In [1]:
import numpy 
import sys 
import nmslib 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.model_selection import train_test_split 
print(sys.version)
print("NMSLIB version:", nmslib.__version__)


In [2]:
# Just a function to read sparse data
def read_data(filename, max_qty = None): 
    row = [] 
    col = [] 
    data = [] 
    read_qty = 0 
    with open(filename,'r') as f:  
        read_num_ft = 0
        for line in f: 
            x = line.replace(':', ' ').strip().split() 
            if (len(x) % 2 != 0):
                raise(Exception('Poorly formated line %d in file %s' % (read_qty + 1, filename)))
            if (len(x) == 0): continue
            for i in range(0, len(x), 2):     
                row.append(read_qty) 
                feat_id = int(x[i])
                read_num_ft = max(read_num_ft, feat_id + 1)
                col.append(feat_id) 
                data.append(1) 

            read_qty = read_qty+1 
            if max_qty != None and read_qty >= max_qty: break
            
    print('Read %d rows, # of features %d' %  (read_qty, read_num_ft))
    ft_mat = csr_matrix((numpy.array(data), (numpy.array(row), numpy.array(col))), 
                         shape=(read_qty, read_num_ft)) 
    return (read_qty, ft_mat)

In [3]:
# Read data points
(all_qty, all_data_matrix) = read_data('../../sample_data/sparse_wiki_5K.txt') 

Read 5000 rows, # of features 100000


In [4]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(all_data_matrix, test_size = 0.1)

In [5]:
print("# of queries %d, # of data points %d"  % (query_matrix.shape[0], data_matrix.shape[0]) )

# of queries 500, # of data points 4500


In [6]:
# For each entry in the sparse matrix, extract a list of IDs and
# convert them to a string. Return a list of such strings.
def matrToStrArray(sparseMatr):
    res = []
    indptr = sparseMatr.indptr
    indices = sparseMatr.indices
    for row in range(sparseMatr.shape[0]):
        arr = [k for k in indices[indptr[row] : indptr[row + 1]]]
        arr.sort()
        res.append(' '.join([str(k) for k in arr]))
    return res

In [7]:
data_matrix_str = matrToStrArray(data_matrix)
query_matrix_str = matrToStrArray(query_matrix)

In [8]:
# Set index parameters
# These are the most important onese
M = 30
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [9]:
# Number of neighbors 
K=100

In [10]:
# Intitialize the library, specify the space, the type of the data and add data points 
# Note that in the GENERIC case, data points are passed as strings!
index = nmslib.init(method='hnsw', space='jaccard_sparse', data_type=nmslib.DataType.OBJECT_AS_STRING) 
index.addDataPointBatch(data_matrix_str) 

4500

In [11]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 30, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}
Indexing time = 5.325021


In [12]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params) 

Setting query-time parameters {'efSearch': 100}


In [13]:
# Querying
query_qty = len(query_matrix_str)
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix_str, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=0.601650 (sec), per query=0.001203 (sec), per query adjusted for thread number=0.004813 (sec)


In [14]:
# Computing gold-standard data 
print('Computing gold-standard data')

gs = []

start = time.time()

query_qty = query_matrix.shape[0]
data_qty = data_matrix.shape[0]

data_row_card = numpy.array([data_matrix.getrow(i).getnnz() for i in range(data_qty)])

for i in range(query_qty):
    #print('Processing query %d/%d' % (i+1, query_qty))
    q = query_matrix.getrow(i)
    query_card = float(q.getnnz())
    inter_card = data_matrix.dot(q.transpose()).toarray().reshape(data_qty)
    # Jaccard distance 1 - intersection_size / union_size
    # union_size = cardinality1 + cardinality2 - intersection_size
    dist_vals = numpy.full(data_qty, 1) - inter_card / (data_row_card + query_card - inter_card)
    tmp = [ (dist_vals[i], i) for i in range(data_qty)]
    tmp.sort()
    gs.append( [tmp[i][1] for i in range(K)] )

end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
brute-force kNN time total=14.332985 (sec), per query=0.028666 (sec)


In [15]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.953200


In [16]:
# Save a meta index and data
index.saveIndex('sparse_jaccard_index.bin', save_data=True)

In [17]:
# Re-intitialize the library, specify the space, the type of the data and add data points 
# Note again that in the GENERIC case, data points are passed as strings!
newIndex = nmslib.init(method='hnsw', space='jaccard_sparse', data_type=nmslib.DataType.OBJECT_AS_STRING)  

In [18]:
# Re-load the index and the data
newIndex.loadIndex('sparse_jaccard_index.bin', load_data=True)

In [19]:
# Setting query-time parameters and querying
print('Setting query-time parameters', query_time_params)
newIndex.setQueryTimeParams(query_time_params)

query_qty = len(query_matrix_str)
start = time.time() 
new_nbrs = newIndex.knnQueryBatch(query_matrix_str, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

Setting query-time parameters {'efSearch': 100}
kNN time total=0.612867 (sec), per query=0.001226 (sec), per query adjusted for thread number=0.004903 (sec)


In [20]:
# Finally computing recall for the new result set
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[i])
  ret_set = set(new_nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.953200
